In [122]:
from sympy import sin, cos, Matrix, Symbol, solve, simplify, pi
import math

In [123]:
# 脚毎の「脚根本ー胴体重心変換行列」を定義
pi=3.14159
Phi = 0
Theta = pi/2.0-0.955316618
Psi = pi/4.0
body = 2.0

R = Matrix([
    [cos(Theta)*cos(Psi), sin(Phi)*sin(Theta)*cos(Psi)-cos(Phi)*sin(Psi), cos(Phi)*sin(Theta)*cos(Psi)+sin(Phi)*sin(Psi)],
    [cos(Theta)*sin(Psi), sin(Phi)*sin(Theta)*sin(Psi)+cos(Phi)*cos(Psi), cos(Phi)*sin(Theta)*sin(Psi)-sin(Phi)*cos(Psi)],
    [-sin(Theta), sin(Phi)*cos(Theta), cos(Phi)*cos(Theta)]
])
# 胴体中心から脚根本までの座標
# BLF
col = Matrix([[body/2.0],
            [body/2.0],
            [-body/2.0]])
row = Matrix([[0, 0, 0, 1]])
T_tmp = R.col_insert(4, col)
convert_BLF_to_center = T_tmp.row_insert(4, row)
print(convert_BLF_to_center)

Matrix([[0.577351193812963, -0.707106312093558, 0.408247795340275, 1.00000000000000], [0.577350427786854, 0.707107250279226, 0.408247253679543, 1.00000000000000], [-0.577349185967282, 0, 0.816497346880513, -1.00000000000000], [0, 0, 0, 1]])


In [124]:
# 回転軸の方向ベクトル
# 元座標からみた相対座標系における回転軸方向のベクトルを表す
class Omega:
    def __init__(self, Lam=0, Mu=0, Nu=0, Phi=0, Theta=0, Psi=0):
        self.Lam, self.Mu, self.Nu, self.Phi, self.Theta, self.Psi = Lam, Mu, Nu, Phi, Theta, Psi
    def subsVal(self, Lam, Mu, Nu):
        self.Lam, self.Mu, self.Nu = Lam, Mu, Nu
    def subsEuler(self, Phi, Theta, Psi):
        self.Phi, self.Theta, self.Psi = Phi, Theta, Psi

# オイラー回転を行った際の, ｚ軸方向ベクトルの回転後ベクトルを計算
def convertEulerToOmega(Phi, Theta, Psi):
        Lam = cos(Phi)*sin(Theta)*cos(Psi)+sin(Phi)*sin(Psi)
        Mu = cos(Phi)*sin(Theta)*sin(Psi)-sin(Phi)*cos(Psi)
        Nu = cos(Phi)*cos(Theta)
        print(Lam, Mu, Nu)
        return Lam, Mu, Nu

# 同時変換行列の導出
def getSimultaneousTransformationXMatrix(omega, theta, L):
    X_roll = Matrix([[1, 0, 0],
                [0, cos(theta), -sin(theta)],
                [0, sin(theta), cos(theta)]])
    RzRyRz = Matrix([
                [cos(omega.Theta)*cos(omega.Psi), sin(omega.Phi)*sin(omega.Theta)*cos(omega.Psi)-cos(omega.Phi)*sin(omega.Psi), cos(omega.Phi)*sin(omega.Theta)*cos(omega.Psi)+sin(omega.Phi)*sin(omega.Psi)],
                [cos(omega.Theta)*sin(omega.Psi), sin(omega.Phi)*sin(omega.Theta)*sin(omega.Psi)+cos(omega.Phi)*cos(omega.Psi), cos(omega.Phi)*sin(omega.Theta)*sin(omega.Psi)-sin(omega.Phi)*cos(omega.Psi)],
                [-sin(omega.Theta), sin(omega.Phi)*cos(omega.Theta), cos(omega.Phi)*cos(omega.Theta)]])
    R = RzRyRz * X_roll
    col = Matrix([[L],
                [0],
                [0]])
    row = Matrix([[0, 0, 0, 1]])
    T_tmp = R.col_insert(4, col)
    T = T_tmp.row_insert(4, row)
    return T

def getSimultaneousTransformationZMatrix(omega, theta, L):
    Z_roll = Matrix([[cos(theta), -sin(theta), 0],
                [sin(theta), cos(theta), 0],
                [0, 0, 1]])
    RzRyRz = Matrix([
                [cos(omega.Theta)*cos(omega.Psi), sin(omega.Phi)*sin(omega.Theta)*cos(omega.Psi)-cos(omega.Phi)*sin(omega.Psi), cos(omega.Phi)*sin(omega.Theta)*cos(omega.Psi)+sin(omega.Phi)*sin(omega.Psi)],
                [cos(omega.Theta)*sin(omega.Psi), sin(omega.Phi)*sin(omega.Theta)*sin(omega.Psi)+cos(omega.Phi)*cos(omega.Psi), cos(omega.Phi)*sin(omega.Theta)*sin(omega.Psi)-sin(omega.Phi)*cos(omega.Psi)],
                [-sin(omega.Theta), sin(omega.Phi)*cos(omega.Theta), cos(omega.Phi)*cos(omega.Theta)]])
    R = RzRyRz * Z_roll
    col = Matrix([[L],
                [0],
                [0]])
    row = Matrix([[0, 0, 0, 1]])
    T_tmp = R.col_insert(4, col)
    T = T_tmp.row_insert(4, row)
    return T

# 合成同時変換行列の導出
def calcCompositeSimultaneousTransformationMatrix(omega, L, theta):
    T_0 = getSimultaneousTransformationXMatrix(omega[0], theta = theta[0], L = L[0])
    T_1 = getSimultaneousTransformationZMatrix(omega[1], theta = theta[1], L = L[1])
    T_2 = getSimultaneousTransformationZMatrix(omega[2], theta = theta[2], L = L[2])
    Theta = Matrix([theta[0], theta[1], theta[2]])
    P_3 = Matrix([[L[3]],
                [0],
                [0],
                [1]])
    P =  convert_BLF_to_center*T_0 * T_1 * T_2 * P_3
    Jacob_T = P.jacobian(Theta)
    # 4行目を消去
    P.row_del(3)
    Jacob_T.row_del(3)
    return Jacob_T, P

In [125]:
theta = [Symbol('theta[0]', real=True), Symbol('theta[1]', real=True), Symbol('theta[2]', real=True)]
euler_0 = [0, 0, 0]
euler_1 = [-pi/2, 0, 0]
euler_2 = [0, 0, 0]
L = [0, Symbol('L1', real=True), Symbol('L2', real=True), Symbol('L3', real=True)]
L = [0.0, 1, 1, 1]
omega = [Omega(), Omega(), Omega()]
omega[0].subsEuler(euler_0[0], euler_0[1], euler_0[2])
omega[1].subsEuler(euler_1[0], euler_1[1], euler_1[2])
omega[2].subsEuler(euler_2[0], euler_2[1], euler_2[2])

for i in range(3):
    convertEulerToOmega(omega[i].Phi, omega[i].Theta, omega[i].Psi)

# ヤコビ行列, 胴体座標
Jacob_T, P = calcCompositeSimultaneousTransformationMatrix(omega, L, theta)
print(Jacob_T)

0 0 1
0 0.999999999999120 1.32679489667753e-6
0 0 1
Matrix([[(-0.99999999999912*(-0.408247795340275*sin(theta[0]) + 0.707106312093558*cos(theta[0]))*sin(theta[1]) + 1.32679489667753e-6*(0.707106312093558*sin(theta[0]) + 0.408247795340275*cos(theta[0]))*sin(theta[1]))*cos(theta[2]) + (-0.99999999999912*(-0.408247795340275*sin(theta[0]) + 0.707106312093558*cos(theta[0]))*cos(theta[1]) + 1.32679489667753e-6*(0.707106312093558*sin(theta[0]) + 0.408247795340275*cos(theta[0]))*cos(theta[1]))*sin(theta[2]) - 0.99999999999912*(-0.408247795340275*sin(theta[0]) + 0.707106312093558*cos(theta[0]))*sin(theta[1]) + 1.32679489667753e-6*(0.707106312093558*sin(theta[0]) + 0.408247795340275*cos(theta[0]))*sin(theta[1]), (-0.707106312092935*sin(theta[0]) - 0.408247795339915*cos(theta[0]))*cos(theta[1]) + (5.4166109143733e-7*sin(theta[0]) - 9.38185046294203e-7*cos(theta[0]))*cos(theta[1]) + (-(-0.707106312092935*sin(theta[0]) - 0.408247795339915*cos(theta[0]))*sin(theta[1]) - (5.4166109143733e-7*sin(theta

In [126]:
now_theta = Matrix([0, -120.0*pi/180.0, 120.0*pi/180.0])

now_pos = P.subs([(theta[0], now_theta[0]), (theta[1], now_theta[1]), (theta[2], now_theta[2])])
print(now_pos.evalf(4))
target_pos = Matrix([1.2, 1.0, -1.3])
repeat = 100
for i in range(repeat):
    # 胴体座標形における足先座標の計算
    ABS_pos = P.subs([(theta[0], now_theta[0]), (theta[1], now_theta[1]), (theta[2], now_theta[2])])
    print(ABS_pos.evalf())
    # 座標フィードバックをもとにして, 脚先の目標方向ベクトルの導出
    target_pos_vector = now_pos+(target_pos-now_pos)*(float(i+1) /  float(repeat)) 
    #print((ABS_pos+target_pos_vector).evalf(4))
    # ヤコビ行列の逆行列の計算
    J = Jacob_T.subs([(theta[0], now_theta[0]), (theta[1], now_theta[1]), (theta[2], now_theta[2])])
    #print(J)
    inv_J = J.inv()
    delta_pos = target_pos_vector - ABS_pos
    delta_theta = inv_J*delta_pos
    # 目標方向ベクトル, 現在関節角度をもとにして目標関節角度を更新
    target_theta = (now_theta + delta_theta).evalf()

    now_theta = target_theta

Matrix([[2.220], [2.220], [-1.159]])
Matrix([[2.21958181065436], [2.21957856753468], [-1.15891649674391]])
Matrix([[2.20936437724746], [2.20738513396460], [-1.16042256354770]])
Matrix([[2.19916958054831], [2.19519122979497], [-1.16183631228235]])
Matrix([[2.18897467352105], [2.18299716637848], [-1.16324929276770]])
Matrix([[2.17877976525502], [2.17080313313191], [-1.16466236089405]])
Matrix([[2.16858485877996], [2.15860913461224], [-1.16607553322643]])
Matrix([[2.15838995597927], [2.14641517473131], [-1.16748881422013]])
Matrix([[2.14819505874297], [2.13422125756521], [-1.16890220838248]])
Matrix([[2.13800016900242], [2.12202738737330], [-1.17031572042998]])
Matrix([[2.12780528873599], [2.10983356861421], [-1.17172935528819]])
Matrix([[2.11761041997339], [2.09763980596168], [-1.17314311808714]])
Matrix([[2.10741556479891], [2.08544610432034], [-1.17455701415377]])
Matrix([[2.09722072535347], [2.07325246884077], [-1.17597104900066]])
Matrix([[2.08702590383486], [2.06105890493389], [-1.1